# Accès aux métadonnées pour classification profil gauche/droit

In [2]:
%load_ext autoreload
%autoreload 2

## Technique qui fonctionne

In [4]:
from config import *
import os
from win32com.client import Dispatch
from utils import withoutExt

PATH = os.path.join(os.environ["INDIV_PATH"], "224", "train", "BF002")
images_name = os.listdir(PATH)
print(images_name)

def get_details(path, image_name):
    shell = Dispatch("Shell.Application")
    _dict = {}
    ns = shell.NameSpace(path)
    for i in ns.Items():
        # Check here with the specific filename
        if str(i) == withoutExt(image_name):
            for j in range(0,49):
                _dict[ns.GetDetailsOf(j,j)] = ns.GetDetailsOf(i,j)

    return _dict["Mots clés"]

for img_name in images_name:
    info = get_details(PATH, img_name)
    if info:
        print(info)
    else:
        print("Empty")

['BF002_2021_@P.Louisy_200608_DSC_0734.JPG.crop.resize224.jpg', 'BF002_2021_@P.Louisy_200608_DSC_0735.JPG.crop.resize224.jpg', 'BF002_2021_@P.Louisy_200608_DSC_0736.JPG.crop.resize224.jpg', 'BF002_2021_@P.Louisy_200608_DSC_0749.JPG.crop.resize224.jpg', 'BF002_2021_@P.Louisy_200608_DSC_0750.JPG.crop.resize224.jpg', 'BF002_2021_@P.Louisy_200608_DSC_0751.JPG.crop.resize224.jpg', 'BF002_2021_@P.Louisy_200608_DSC_0752.JPG.crop.resize224.jpg', 'BF002_2021_@P.Louisy_200608_DSC_0782.JPG.crop.resize224.jpg', 'BF002_2021_@P.Louisy_200608_DSC_0823.JPG.crop.resize224.jpg']
droite
droite
gauche
gauche
gauche
gauche
gauche
droite
gauche


## Technique qui ne fonctionne pas

In [ ]:
# import exif
# import piexif
# from PIL import Image
# from PIL.ExifTags import TAGS

# def get_exif1(path):
#   i = Image.open(path)
#   info = i._getexif()
#   return {TAGS.get(tag): value for tag, value in info.items()}

# def get_exif2(path):
#     with open(path, 'rb') as img_file:
#         return exif.Image(img_file).list_all()
      
# def get_exif3(path):
#     exif_dict = piexif.load(path)
#     output = {}
#     for ifd in exif_dict:
#         if exif_dict[ifd]:
#             for tag in exif_dict[ifd]:
#                 tag_name = piexif.TAGS[ifd][tag]["name"]
#                 tag_value = exif_dict[ifd][tag]
#                 # Avoid print a large value, just to be pretty
#                 if isinstance(tag_value, bytes):
#                     tag_value = tag_value[:10]
#                 output[tag_name] = tag_value
#     return output

## Création du Datatset

In [30]:
from config import *
import os
from win32com.client import Dispatch
from .utils import withoutExt
from tqdm import tqdm
import shutil

In [ ]:
def getDetails(path, image_name):
    shell = Dispatch("Shell.Application")
    _dict = {}
    ns = shell.NameSpace(path)
    for i in ns.Items():
        # Check here with the specific filename
        if str(i) == withoutExt(image_name):
            for j in range(0,49):
                _dict[ns.GetDetailsOf(j,j)] = ns.GetDetailsOf(i,j)

    return _dict["Mots clés"]

In [18]:
PATH = os.environ["INDIV_PATH"]
PATH = os.path.join(PATH, "224")

left = {}
right = {}

for set in os.listdir(PATH):
    print(set)
    for indiv in tqdm(os.listdir(os.path.join(PATH, set))):
        for image_name in os.listdir(os.path.join(PATH, set, indiv)):
            key = getDetails(os.path.join(PATH, set, indiv), image_name)
            if not key:
                continue
            if key == "gauche":
                left[indiv] = left.get(indiv, []) + [image_name]
            elif key == "droite":
                right[indiv] = right.get(indiv, []) + [image_name]

test


100%|██████████| 342/342 [01:05<00:00,  5.22it/s]


train


100%|██████████| 308/308 [11:01<00:00,  2.15s/it] 


In [19]:
tt_left = sum([len(images) for images in left.values()])
print("Tous les profils gauches touvés : ", tt_left==1716)
tt_right = sum([len(images) for images in right.values()])
print("Tous les profils droits touvés : ", tt_right==1906)

Tous les profils gauches touvés :  True
Tous les profils droits touvés :  True


In [36]:
IN_PATH = os.environ["DATASET_PATH"]
OUT_PATH = os.environ["PROFIL_PATH"]

if not os.path.isdir(OUT_PATH):
    os.mkdir(OUT_PATH)
    
if not os.path.isdir(os.path.join(OUT_PATH, "left")):
    os.mkdir(os.path.join(OUT_PATH, "left"))
    
if not os.path.isdir(os.path.join(OUT_PATH, "right")):
    os.mkdir(os.path.join(OUT_PATH, "right"))
    
parts = {"left": left, "right": right}
    
for part_name, part in parts.items():
    for indiv in part.keys():
        for image_name in tqdm(part[indiv]):
            if not os.path.isfile(os.path.join(OUT_PATH, part_name, indiv, image_name)):
                in_file = os.path.join(IN_PATH, indiv, image_name)
                out_file = os.path.join(OUT_PATH, part_name, image_name)
                shutil.copyfile(in_file, out_file)

100%|██████████| 7/7 [00:00<00:00, 1166.57it/s]
